# Calcular ganancia productos de Emprende Tienda :)

In [ ]:
# Instalaciones

import requests
!pip install tika
from tika import parser
import pandas as pd
import random
import numpy as np
import time
from google.colab import files
%pip install bs4
from bs4 import BeautifulSoup
!pip install selenium
!apt-get update 
!apt install chromium-chromedriver
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
options = webdriver.ChromeOptions() # Usamos chrome, se podria usar otro.
options.add_argument('--headless') # Chromium sin interfaz grafica
options.add_argument('--no-sandbox') # Seguridad
options.add_argument('--disable-dev-shm-usage') # configuracion de linux
options.add_argument('--user-agent=""Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.157 Safari/537.36""') # user agent

# Ver cuanto toma
start = time.time()

# Obtener nombres, precios, links y codigos desde ET

# Instanciamos el webdriver
driver = webdriver.Chrome('chromedriver',options=options)

# Hacemos que vaya a la url
url = 'https://royalsuplementos.empretienda.com.ar/'
driver.get(url)

# Click en boton Ver Mas
for i in range(10):
  try:
    driver.execute_script("window.scrollTo(10, document.body.scrollHeight);")
    driver.save_screenshot(f'infinite_page_{i+1}.screenshot.png')
    element = driver.find_element_by_xpath('//*[@id="products_feed-btn"]')
    driver.execute_script("arguments[0].click();", element)
  except:
    try:
      driver.execute_script("window.scrollTo(10, document.body.scrollHeight);")
      driver.save_screenshot(f'infinite_page_{i+1}.screenshot.png')
      element = driver.find_element_by_xpath('/html/body/section/div/div[2]/a')
      driver.execute_script("arguments[0].click();", element)
    except:
      continue

# Nombres
nombres = []
nombres = driver.find_elements(By.CSS_SELECTOR, 'h3')
nombres = [nombre.text for nombre in nombres]

# Precios
precios = []
precios = driver.find_elements(By.CSS_SELECTOR, 'p')
precios = [precio.text for precio in precios]
precios = [precio for precio in precios if '$' in precio]

# Links
links = []
links = driver.find_elements_by_css_selector('h3 a')
links = [link.get_attribute('href') for link in links]

# Codigo
contador = 0
codigos = []
for url in links:
    contador +=1
    print(f"Obteniendo codigo de producto nro {contador} de {len(links)}")
    try:
        driver.get(url)
        cod = driver.find_elements(By.CSS_SELECTOR, 'div p')
        cod = [d.text for d in cod]
        cod = [d for d in cod if 'CODIGO_PRODUCTO' in d]
        cod = cod[0].split('=')[-1]
        cod = cod.replace(' ','')
        codigos.append(int(cod))
    except:
        codigos.append("Sin codigo")
        continue
codigos1 = codigos.copy()

# Eliminacion de duplicados

# Nombres
new_nombres = []
indices = []
for i,v in enumerate(nombres):
  if v not in new_nombres:
    new_nombres.append(v)
    indices.append(i)
nombres = new_nombres.copy()

# Links
new_links = []
for i,v in enumerate(links):
  if i in indices:
    new_links.append(v)
links = new_links.copy()

# Precios
new_precios = []
for i,v in enumerate(precios):
  if i in indices:
    new_precios.append(v)
precios = new_precios.copy()
precios = [i.replace('$', '') for i in precios]
precios = [i.split(',')[0] for i in precios]
precios = [int(i.replace('.', '')) for i in precios]

# Sustituto de descripciones (productos sin codigo en descripcion)
lista = ['Sin codigo']*(len(nombres) - len(codigos))
codigos.extend(lista)

# Print de productos sin codigo:
print("Productos sin codigo en su descripcion: ")
for i,v in enumerate(nombres):
  if codigos[i] == 'Sin codigo':
    print(v)






# Obtener los codigos, precios y articulos de DYD

# Lectura
raw = parser.from_file('preferencial.pdf')
df = (raw['content'])
df = df.replace('\n',',')
df = df.replace(',,',',')
df = df.replace(',,,,,,,,,,,,,,,,,,,,','')

# Reemplazo de las comas en los precios (para luego dividir los productos por coma)
for i,v in enumerate(df):
  if i < len(df)-3:
    if v == ',':
      if df[i+2] == '0':
        df = df[:i] + '.' + df[i + 1:]

      try:
        valor1 = int(df[i+1])
        if type(valor1) == int:
          df = df[:i] + '.' + df[i + 1:]
      except:
        continue

      try:
        valor2 = int(df[i+2])
        if type(valor2) == int:
          df = df[:i] + '.' + df[i + 1:]
      except:
          continue

# Split
df = df.split(',')

# Obtener los codigos, precios y articulos
codigos_DYD = []
articulos_DYD = []
precios_DYD = []
for producto in df:
  if '$' in producto:
    articulos_DYD.append(producto)
    producto = producto.split(' ')
    precios_DYD.append(int(producto[-1]))
    codigo = 0
    for cod in producto:
      try:
        codigo = int(cod)  
        break
      except:
        continue
    if codigo > 0:
      codigos_DYD.append(codigo)
    else:
      codigos_DYD.append('Codigo no encontrado')







# PRODUCTOS QUE NO ESTAN EN DYD

# Funcion agregadora de productos
# Molde
nombres_otros = []
precios_otros = []
codigos_otros = []

def creador_producto(nombre, costo, codigo):
  ok = True
  if nombre in nombres_otros or codigo in codigos_otros:
    ok = False
  if ok == True:
    nombres_otros.append(nombre)
    precios_otros.append(costo)
    codigos_otros.append(codigo)
  else:
    print(f'Error: El nombre "{nombre}" o el codigo {codigo} ya existe/n en la lista')  



# Productos fuera dyd: Poner entre parentesis en el siguiente orden y separado por coma: 'nombre del producto', costo, codigo generado.
# Luego, ir a emprende tienda, y al agregar el producto, al final de la descripcion poner "CODIGO_PRODUCTO = *codigo generado*"

# 1) 
creador_producto('Fuark Remera Oversized Stand Out White', 1400, 2941)

# 2) 
creador_producto('Jogging Human Black - Fuark', 2900, 9850)

# 3) 
creador_producto('Musculosa Reeb - Fuark', 1080, 3971)

# 4) 
creador_producto('Hoodie Athletics Black - Fuark', 1100, 7974)

# 5) 
creador_producto('Remera Performance Cross Black - Fuark', 1080, 1621)

# 6) 
creador_producto('Short Rustik Spikes Pink - Fuark', 2100, 9706)

# 7) 
creador_producto('Short Rustik Spikes Black - Fuark', 2100, 9219)

# 8) 
creador_producto('Musculosa Reeb Flag Blue - Fuark', 1080, 6389)

# 9) 
creador_producto('BSN Syntha 6 ISOLATE 2 Libras', 9000, 8340)

# 10) 
creador_producto('Optimun Nutrition GOLD STANDARD WHEY 5 Libras (2.27 kg.)', 15000, 8800)

# 11) 
creador_producto('Optimun Nutrition GOLD STANDARD WHEY 2 Libras', 7434, 2900)

# 12) 
creador_producto('BSN - Syntha-6 Edge x 4Lbs (1.64 kg) - Vainilla', 15000, 9033)













# Determinar largo del DF
codigos_existentes = [i for i in codigos if i != 'Sin codigo']

largo = len(nombres)
df = pd.DataFrame(np.array([[0]*largo,[0]*largo,[0]*largo,[0]*largo,
                            [0]*largo,[0]*largo]).T,
                   columns=['Nombre_ET','Ganancia', 'Precio_Venta', 'Costo', 'Nombre_Proveedor','Codigo'])

# Aparear por codigos y generar DF

for i,v in enumerate(codigos):
  for i2,v2 in enumerate(codigos_DYD):
    # Esta en DYD
    try:
      if v == v2:
        df['Nombre_ET'][i] = nombres[i]
        df['Codigo'][i] = codigos_DYD[i2]
        df['Nombre_Proveedor'][i] = articulos_DYD[i2]
        df['Costo'][i] = precios_DYD[i2]
        df['Precio_Venta'][i] = precios[i]
        try:
          df['Ganancia'][i] = precios[i] - precios_DYD[i2]
        except:
          df['Ganancia'] = 'Sin dato'
      else:

        # Lo compramos a otro distribuidor
        for i3,v3 in enumerate(codigos_otros):
          if v == v3:
            df['Codigo'][i] = codigos[i]
            df['Nombre_ET'][i] = nombres[i]
            df['Nombre_Proveedor'][i] = nombres_otros[i3]
            df['Costo'][i] = precios_otros[i3]
            df['Precio_Venta'][i] = precios[i]
            try:
              df['Ganancia'][i] = precios[i] - precios_otros[i3]
            except:
              df['Ganancia'] = 'Sin dato'
    except:
      continue


df = df.sort_values('Ganancia')
stop = time.time()
print(f"Este codigo tardo: {stop - start} segundos en ejecutarse")
# Eliminado de filas inutiles
df = df[df.Nombre_ET != 0]
df.head(60)

     |████████████████████████████████| 127 kB 5.2 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.9
    Uninstalling urllib3-1.26.9:
      Successfully uninstalled urllib3-1.26.9
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
selenium 4.1.5 requires urllib3[secure,socks]~=1.26, but you have urllib3 1.25.11 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


  Using cached urllib3-1.26.9-py2.py3-none-any.whl (138 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.25.11
    Uninstalling urllib3-1.25.11:
      Successfully uninstalled urllib3-1.25.11
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.9 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Reading package lists... Done
Reading package lists... Done
Building dependency t

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:45: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:51: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:69: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead


Obteniendo codigo de producto nro 1 de 90
Obteniendo codigo de producto nro 2 de 90
Obteniendo codigo de producto nro 3 de 90
Obteniendo codigo de producto nro 4 de 90
Obteniendo codigo de producto nro 5 de 90
Obteniendo codigo de producto nro 6 de 90
Obteniendo codigo de producto nro 7 de 90
Obteniendo codigo de producto nro 8 de 90
Obteniendo codigo de producto nro 9 de 90
Obteniendo codigo de producto nro 10 de 90
Obteniendo codigo de producto nro 11 de 90
Obteniendo codigo de producto nro 12 de 90
Obteniendo codigo de producto nro 13 de 90
Obteniendo codigo de producto nro 14 de 90
Obteniendo codigo de producto nro 15 de 90
Obteniendo codigo de producto nro 16 de 90
Obteniendo codigo de producto nro 17 de 90
Obteniendo codigo de producto nro 18 de 90
Obteniendo codigo de producto nro 19 de 90
Obteniendo codigo de producto nro 20 de 90
Obteniendo codigo de producto nro 21 de 90
Obteniendo codigo de producto nro 22 de 90
Obteniendo codigo de producto nro 23 de 90
Obteniendo codigo de

2022-05-13 20:17:35,533 [MainThread  ] [INFO ]  Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server/1.24/tika-server-1.24.jar to /tmp/tika-server.jar.


Productos sin codigo en su descripcion: 
Muscletech NITRO TECH WHEY GOLD 1 Kg - Strawberry Shortcake
Hoch Sport Proteína Whey 3 Kg Materias Primas Importadas
Mervick - Multivitaminico y Mineral x 120 comprimidos
Nutrilab PROTEÍNA WHEY PRO x 1 KG


2022-05-13 20:17:36,043 [MainThread  ] [INFO ]  Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server/1.24/tika-server-1.24.jar.md5 to /tmp/tika-server.jar.md5.
2022-05-13 20:17:36,364 [MainThread  ] [WARNI]  Failed to see startup log message; retrying...
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:282: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:283: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be 

Este codigo tardo: 245.5662829875946 segundos en ejecutarse


,Nombre_ET,Ganancia,Precio_Venta,Costo,Nombre_Proveedor,Codigo
7,Creatina Powder 200 Grs - Universal Nutrition,-1970,5145,7115,UNIVERSAL NUT… 215 Creatina Powder X 200 + 200...,215
46,Nutrilab ONE MULTIVITAMIN ACTIVATOR x 120 comp...,669,1517,848,Nutrilab 92 One 120 Comp. $ 848,92
54,Nutrilab CREATINA SHOCK x 300 Gr,722,2260,1538,Nutrilab 974 Creatine (10 G./servicio. Sabor: ...,974
49,Nutrilab PROTEÍNA WHEY PRO x 3 Kg,807,5025,4218,Nutrilab 696 Whey Pro 3 Kg 2.0 Vainilla $ 4218,696
23,Gentech BARRA PROTEÍCA ENERGETICA x 20 Unidades,836,2410,1574,Gentech 893 Iron Bar X 20 Unid. Chocolate $ 1574,893
47,Nutrilab NITRIC OXIDE x 180 Comprimidos (2x1),960,3244,2284,Nutrilab 88 Nitric Oxide (180 Comp. 2x1) $ 2284,88
63,X Body Evolution - Bcaa 300 gr. - Extra Amino,1004,2728,1724,X-BODY EVOLUTI… 2244 Bcaa Limon X 300 G. $ 1724,2244
52,Nutrilab BCAA x 300 Gr,1037,2404,1367,Nutrilab 866 Bcaa X 300 Gr (10 G./servicio. Sa...,866
57,Creatina - X-Body Evolution - Creatina Microni...,1116,3990,2874,X-BODY EVOLUTI… 2241 Creatina Micronizada X 30...,2241
9,Fuark Remera Oversized Stand Out White,1120,2520,1400,Fuark Remera Oversized Stand Out White,2941


In [ ]:
# <----- Apretar Play para descargarlo como excel
df.to_csv('Ganancias_ET.csv')
files.download("Ganancias_ET.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Agregar un producto:

**Si esta en DYD:** 
1. Agregar en descripcion del producto en ET "CODIGO_PRODUCTO = " junto con el codigo tal cual figura en la lista de DYD.

**Si NO es un producto de DYD:**
1. Correr todo el codigo.
2. Correr el codigo de debajo para generar un codigo de producto.
3. Ir a la mitad del codigo de arriba y agregar "creador_producto(x,y,z)" siendo x = nombre producto (tal cual esta en ET), y = costo (o precio de compra), z = codigo generado (con la funcion de abajo).
4. Agregar en descripcion del producto en ET "CODIGO_PRODUCTO = " junto con el codigo generado con el codigo de abajo.

In [ ]:
# Generar un codigo para productos fuera de DYD
cond = True
while cond == True:
  n = random.randint(0,10000)
  if n not in codigos_DYD and n not in codigos_otros:
    cond = False
print(f"Codigo generado: {n}")

Codigo generado: 9033
